In [1]:
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Dense, Dropout, Flatten, Conv2D, Input, Add, \
                         Activation, ZeroPadding2D, BatchNormalization, \
                         AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.initializers import glorot_uniform
import tensorflow.keras as tk

Primo blocco conv, NON è un residual block!\
Output del layer shape = ??

In [87]:
def conv1(X, block="conv1", stage=2):
            
        # defining name basis
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        
        X_input = X
        # First component of main path
        X = Conv2D(64, kernel_size = (3,6), strides = (2,1),
                name = conv_name_base + '2a',
                #nchannels??
                # data_format="channels_first", 
                # kernel_initializer = glorot_uniform(seed=0)
                )(X)
        X = BatchNormalization(axis = 1, name = bn_name_base + '2a')(X)
        X = Activation('relu')(X)
        
        # Second component of main path
        X = Conv2D(64, kernel_size = (3,3), strides = (2,2),
                name = conv_name_base + '2b',
                #nchannels?? 
                # kernel_initializer = glorot_uniform(seed=0)
                )(X)
        X = BatchNormalization(axis=1, name = bn_name_base + '2b')(X)
        X = Activation('relu')(X)

        X = MaxPooling2D((2,2), strides=(1, 1))(X)
        
        return X

In [88]:
# X_input = Input(shape=(219,122,2)) 

# res1 = conv1(X_input)
# res1.summary()

Secondo blocco conv: Residual block!

In [89]:
def conv2x(X, stride, filters, block,stage=2):
        # defining name basis
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'

        #filters for each ==> there are two filters because 4 layer 
        #                     --> the first 2 and the last 2 have the same f 
        f1,f2 = filters
        s = stride
        #skip path for the residual part
        X_shortcut = Conv2D(f2, kernel_size = (3,3), strides = (s,s),
                name = conv_name_base + '2',
                # kernel_initializer = glorot_uniform(seed=0)
                )(X)

        X_shortcut = BatchNormalization(axis=1, name = bn_name_base + '2')(X_shortcut)

        # First component of main path
        X = Conv2D(f1, kernel_size = (1,1), strides = (1,1),
                name = conv_name_base + '2a',
                #nchannels?? 
                # kernel_initializer = glorot_uniform(seed=0)
                )(X)
        X = BatchNormalization(axis = 1, name = bn_name_base + '2a')(X)
        X = Activation('relu')(X)

        # Second component of main path
        X = Conv2D(f1, kernel_size = (3,3), strides = (s,s),
                name = conv_name_base + '2b',
                #nchannels?? 
                # kernel_initializer = glorot_uniform(seed=0)
                )(X)
        X = BatchNormalization(axis=1, name = bn_name_base + '2b')(X)
        X = Activation('relu')(X)
        
        # Second component of main path
        X = Conv2D(f2, kernel_size = (1,1), strides = (1,1),
                name = conv_name_base + '2c',
                #nchannels?? 
                # kernel_initializer = glorot_uniform(seed=0)
                )(X)
        X = BatchNormalization(axis=1, name = bn_name_base + '2c')(X)
        X = Activation('relu')(X)

        X = Add()([X, X_shortcut])
        out = Activation('relu')(X)

        return out


In [90]:
# X_input = Input(shape=( 24, 26, 256)) 

# res2 = conv2x(X_input, stride=2, filters=[128,512], block="a")
# res2.summary()

In [95]:
def ResNetJ(feature, lr_power=-3.0, lr_decay=0.0):
    """
    Implementation of the popular ResNet50 the following architecture:
    Arguments:
        feature is a string which allow two input values:
        --> 'vertex' or 'energy'
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(shape=(230,124,2))
    
    # Stage 1
    X = conv1(X_input, block="conv1")
    # Stage2
    X = conv2x(X, stride=1, filters=[32,128], block="conv2x_1")
    X = conv2x(X, stride=1, filters=[32,128], block="conv2x_2")
    X = conv2x(X, stride=1, filters=[32,128], block="conv2x_3")

    # Stage 3
    # X = conv2x(X, stride=2, filters=[64,256], block="conv3x_1")
    X = conv2x(X, stride=1, filters=[64,256], block="conv3x_2")
    X = conv2x(X, stride=1, filters=[64,256], block="conv3x_3")
    X = conv2x(X, stride=1, filters=[64,256], block="conv3x_4")

    # Stage 4 
    # X = conv2x(X, stride=2, filters=[128,512], block="conv4x_1")
    X = conv2x(X, stride=1, filters=[128,512], block="conv4x_2")
    X = conv2x(X, stride=1, filters=[128,512], block="conv4x_3")
    X = conv2x(X, stride=1, filters=[128,512], block="conv4x_4")
    X = conv2x(X, stride=1, filters=[128,512], block="conv4x_5")
    X = conv2x(X, stride=1, filters=[128,512], block="conv4x_6")

    # Stage 5
    # X = conv2x(X, stride=2, filters=[256,1024], block="conv5x_1")
    X = conv2x(X, stride=1, filters=[256,1024], block="conv5x_2")
    X = conv2x(X, stride=1, filters=[256,1024], block="conv5x_3")


    # AVGPOOL 
    X = AveragePooling2D((8,8), name='avg_pool')(X)
    # Flatten
    X = Flatten()(X)
    X = Dense(512, name='first_dense',  kernel_initializer=glorot_uniform(seed=0))(X)
    X = Dense(100, name='second_dense', kernel_initializer=glorot_uniform(seed=0))(X)

    # Output 
    if(feature=="energy"):
        X = Dense(1, name='fc_outputs', kernel_initializer=glorot_uniform(seed=0))(X)
    elif(feature=="vertex"):
        X = Dense(3, name='fc_outputs', kernel_initializer=glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name = 'ResNetJ')
    
    # Compile model
    learning_rate = 10.0**(lr_power)
    opt = tk.optimizers.Adam(lr=learning_rate, beta_1 = 0.9, beta_2 = 0.999 )

    model.compile(loss="mean_squared_error", optimizer=opt, metrics=['accuracy'])
    
    return model    

In [96]:
res = ResNetJ(feature="vertex")
res.summary()

Model: "ResNetJ"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_30 (InputLayer)          [(None, 230, 124, 2  0           []                               
                                )]                                                                
                                                                                                  
 res2conv1_branch2a (Conv2D)    (None, 114, 119, 64  2368        ['input_30[0][0]']               
                                )                                                                 
                                                                                                  
 bn2conv1_branch2a (BatchNormal  (None, 114, 119, 64  456        ['res2conv1_branch2a[0][0]']     
 ization)                       )                                                           

C:\Users\Pietro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


                                                                                                  
 activation_368 (Activation)    (None, 47, 50, 64)   0           ['bn2conv3x_3_branch2a[0][0]']   
                                                                                                  
 res2conv3x_3_branch2b (Conv2D)  (None, 45, 48, 64)  36928       ['activation_368[0][0]']         
                                                                                                  
 bn2conv3x_3_branch2b (BatchNor  (None, 45, 48, 64)  180         ['res2conv3x_3_branch2b[0][0]']  
 malization)                                                                                      
                                                                                                  
 activation_369 (Activation)    (None, 45, 48, 64)   0           ['bn2conv3x_3_branch2b[0][0]']   
                                                                                                  
 res2conv3

In [24]:
def f(n,k,s):
    n_out = (n-k)/s
    return n_out

In [43]:
f(26,5,1)

21.0